<h2>Practical Tasks. Lesson 3<h2>

<h3>Import modules<h3>

In [92]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

<h3>Common code<h3>

In [93]:
def standard_scale(x):
    return (x - x.mean()) / x.std()

In [94]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [95]:
# создаем датасет и стандартизируем значения фичей
X = np.array(
            [[   1,    1,  500,    1],
            [   1,    1,  700,    1],
            [   1,    2,  750,    2],
            [   1,    5,  600,    1],
            [   1,    3, 1450,    2],
            [   1,    0,  800,    1],
            [   1,    5, 1500,    3],
            [   1,   10, 2000,    3],
            [   1,    1,  450,    1],
            [   1,    2, 1000,    2]]
            )

X_st = X.copy().astype(np.float64)
for i in range(1, X_st.shape[1]):
    X_st[:, i] = standard_scale(X_st[:, i])     

In [96]:
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

In [97]:
# перемешивание
np.random.seed(12)
shuffle_index = np.random.permutation(X_st.shape[0])
X_shuffled, y_shuffled = X[shuffle_index], y[shuffle_index]

# разбивка на обучающую и тестовую выборки
train_proportion = 0.5
train_test_cut = int(len(X) * train_proportion)

X_train, X_test, y_train, y_test = \
    X_shuffled[:train_test_cut], \
    X_shuffled[train_test_cut:], \
    y_shuffled[:train_test_cut], \
    y_shuffled[train_test_cut:]

print("Размер массива признаков обучающей выборки", X_train.shape)
print("Размер массива признаков тестовой выборки", X_test.shape)
print("Размер массива ответов для обучающей выборки", y_train.shape)
print("Размер массива ответов для тестовой выборки", y_test.shape)

Размер массива признаков обучающей выборки (5, 4)
Размер массива признаков тестовой выборки (5, 4)
Размер массива ответов для обучающей выборки (5,)
Размер массива ответов для тестовой выборки (5,)


<h3>Task 1<h3>

In [98]:
def calc_logloss(y, y_pred):
    eps = 1e-16
    y_pred_temp = np.clip(y_pred, eps, 1 - eps) # clip zero and one probability values
    err = - np.mean(y * np.log(y_pred_temp) + (1.0 - y) * np.log(1.0 - y_pred_temp))
    return err

<h3>Task 2<h3>

In [99]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        # err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        # if i % (iterations / 10) == 0:
            # print(i, W, err)
    return W

In [100]:
# generate arrays of eta's and iteration numbers to find optimal values
N = 15
N_min, N_max = 100, 10000 # min max number of iterations
eta_min, eta_max = 1e-5, 1e-3
N_arr = [int(N_min + i*(N_max - N_min) / N) for i in range(N + 1)] # generate N+1 int numbers between N_min, N_max
eta_arr = list(np.linspace(eta_min, eta_max, N + 1))

In [101]:
# calculate logloss at different values of parameters and write it into logloss_res array
logloss_res = np.zeros((N + 1, N + 1)) # matrix of results
for i, eta in enumerate(eta_arr):
    for j, N in enumerate(N_arr):
        W = eval_model(X_train, y_train, N, eta)
        z = np.dot(X_train, W)
        y_pred = sigmoid(z)
        logloss_res[i, j] = calc_logloss(y_train, y_pred)

In [102]:
eta_opt_idx, N_opt_idx = np.unravel_index(logloss_res.argmin(), logloss_res.shape) # find indexes of optimal parameters
print(f"Min logloss={logloss_res[eta_opt_idx, N_opt_idx]:.2f} at eta={eta_arr[eta_opt_idx]:.5f}, iterations={N_arr[N_opt_idx]}")

Min logloss=0.67 at eta=0.00008, iterations=4720


<h3>Task 3<h3>

In [103]:
def calc_pred_proba(X, W):
    # calculate probabilities
    return sigmoid(np.dot(X, W))

In [104]:
# find optimal weights and calculate probabilities
W_opt = eval_model(X_train, y_train, N_arr[N_opt_idx], eta_arr[eta_opt_idx])
print(f"Weights vector: {W_opt}")
print(calc_pred_proba(X_train, W_opt))

Weights vector: [ 0.50060061 -0.23108408 -0.00236107  1.52491793]
[0.53411285 0.67523383 0.12372447 0.64882997 0.36207597]


<h3>Task 4<h3>

In [105]:
def calc_pred(X, W, threshold = 0.5):
    # calculate class value
    probs = calc_pred_proba(X, W)
    return np.where(probs > threshold, 1, 0)

<h3>Task 5<h3>

In [106]:
def accuracy(y_train, y_test, X_train, X_test, w):
    y_predicted_train = calc_pred(X_train, W)
    y_predicted_test = calc_pred(X_test, W)
    
    train_accuracy = np.mean(y_predicted_train == y_train) * 100.0
    test_accuracy = np.mean(y_predicted_test == y_test) * 100.0

    print(f"Точность на обучающей выборке: {train_accuracy:.3f}")
    print(f"Точность на тестовой выборке: {test_accuracy:.3f}")

accuracy(y_train, y_test, X_train, X_test, W_opt)

Точность на обучающей выборке: 40.000
Точность на тестовой выборке: 60.000


Вывод: нет признака переобучения

In [107]:
def error_matrix(X, y, W):
    # naive realization
    y_predicted = calc_pred(X, W)
    TP, FP, FN, TN = 0, 0, 0, 0
    for y, y_pred in zip(y, y_predicted):
        if y == y_pred:
            if y: TP += 1
            else: TN += 1
        else:
            if y: FP += 1
            else: FN += 1
    return np.array([[TP, FP], [FN, TN]])

error_matrix_train = error_matrix(X_train, y_train, W_opt)
error_matrix_test = error_matrix(X_test, y_test, W_opt)
print(error_matrix_train)
print(error_matrix_test)

[[1 1]
 [2 1]]
[[3 0]
 [1 1]]


In [108]:
def metrics_calculation(error_matrix):
    TP, FP, FN, TN = error_matrix.flatten()
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    F1 = 2 * recall * precision / (recall + precision)
    return precision, recall, F1

precision_train, recall_train, F1_train = metrics_calculation(error_matrix_train)
precision_test, recall_test, F1_test = metrics_calculation(error_matrix_test)
print(f"precision_train={precision_train:.2f}, recall_train={recall_train:.2f}, F1_train={F1_train:.2f}")
print(f"precision_test={precision_test:.2f}, recall_test={recall_test:.2f}, F1_test={F1_test:.2f}")

precision_train=0.50, recall_train=0.33, F1_train=0.40
precision_test=1.00, recall_test=0.75, F1_test=0.86


<h3>Task 6<h3>

Вывод: на тестовой выборке показатели качества выше, признаков переобучения нет. Вывод является предварительным. Ввиду малости выборки объекты разных класов могли неравномерно распределиться между обучающим и тестовым датасетом. Вследствие этого возможно переобучение при применении алгоритма на более крупных выборках